In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import os
import warnings

from scipy.optimize import curve_fit
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from plotly.subplots import make_subplots

warnings.simplefilter(action='ignore', category=FutureWarning)

Código para transformar los datos en formato .txt a .csv para el tratamiento con pandas

In [44]:
# Ruta del directorio que contiene los archivos de texto
ruta = 'D:/Estiven/Datos/Proyectos/fisica_experimental_lab/6.0-final-project-magnetic-field-detection'

for archivo_txt in os.listdir(ruta):
    if archivo_txt.endswith('.txt'):
        # Obtener el nombre del archivo sin la extensión
        nombre_archivo = os.path.splitext(archivo_txt)[0]
        
        # Leer el archivo y guardarlo como csv
        archivo = pd.read_csv(os.path.join(ruta, archivo_txt), encoding='latin-1')
        archivo.to_csv(os.path.join(ruta, f'{nombre_archivo}.csv'), index=None)

Función para transformar los datos y normalizar la base de datos

In [45]:
def transformacion_columnas(ruta: str):
    for archivo_csv in os.listdir(ruta):
        if archivo_csv.endswith('.csv'):
            nombre_archivo = os.path.splitext(archivo_csv)[0]
            datos = pd.read_csv(archivo_csv, names=['estado_digital', 'valor_sensor', 'voltaje', 'campo_gauss', 'campo_militeslas'])

            # Recorrer columnas y aplicar transformación a los valores
            for columna in datos:
                datos[columna] = datos[columna].apply(lambda x: x.split(':')[-1])

            datos.to_csv(f'{ruta}/{nombre_archivo}.csv', index=None, sep=';')

transformacion_columnas(ruta)

In [12]:
# Lectura de datos
datos_iman1 = pd.read_json('{"estado_digital":{"0":0,"1":0,"2":0,"3":0},"valor_sensor":{"0":533,"1":536,"2":551,"3":586},"voltaje":{"0":260.508,"1":261.975,"2":269.306,"3":286.413},"campo_gauss":{"0":0.00543,"1":0.01357,"2":0.0543,"3":0.14934},"campo_militeslas":{"0":0.00054,"1":0.00136,"2":0.00543,"3":0.01493},"distancia_metros":{"0":0.15,"1":0.1,"2":0.05,"3":0.01}}')
datos_iman2 = pd.read_json('{"estado_digital":{"0":0,"1":0,"2":0,"3":0,"4":0},"valor_sensor":{"0":536,"1":539,"2":546,"3":559,"4":580},"voltaje":{"0":261.975,"1":263.441,"2":266.862,"3":273.216,"4":283.48},"campo_gauss":{"0":0.01629,"1":0.02444,"2":0.04345,"3":0.07875,"4":0.13577},"campo_militeslas":{"0":0.00163,"1":0.00244,"2":0.00434,"3":0.00787,"4":0.01358},"distancia_metros":{"0":0.1,"1":0.08,"2":0.06,"3":0.04,"4":0.02}}')
datos_iman3 = pd.read_json('{"estado_digital":{"0":0,"1":0,"2":0,"3":0,"4":0},"valor_sensor":{"0":528,"1":527,"2":526,"3":525,"4":523},"voltaje":{"0":258.065,"1":257.576,"2":257.087,"3":256.598,"4":255.621},"campo_gauss":{"0":-0.00543,"1":-0.00814,"2":-0.01086,"3":-0.01357,"4":-0.019},"campo_militeslas":{"0":-0.00054,"1":-0.00081,"2":-0.00109,"3":-0.00136,"4":-0.0019},"distancia_metros":{"0":0.05,"1":0.04,"2":0.03,"3":0.02,"4":0.01}}')
datos_iman4 = pd.read_json('{"estado_digital":{"0":0,"1":0,"2":0,"3":0},"valor_sensor":{"0":531,"1":533,"2":537,"3":547},"voltaje":{"0":259.531,"1":260.508,"2":262.463,"3":267.351},"campo_gauss":{"0":0.00272,"1":0.00815,"2":0.01901,"3":0.04616},"campo_militeslas":{"0":0.00027,"1":0.00081,"2":0.0019,"3":0.00462},"distancia_metros":{"0":0.05,"1":0.04,"2":0.03,"3":0.02}}')
datos_iman1

,estado_digital,valor_sensor,voltaje,campo_gauss,campo_militeslas,distancia_metros
0,0,533,260.508,0.00543,0.00054,0.15
1,0,536,261.975,0.01357,0.00136,0.10
2,0,551,269.306,0.05430,0.00543,0.05
3,0,586,286.413,0.14934,0.01493,0.01


In [13]:
# Transformaciones y unión de los datos
datos_totales = pd.concat([datos_iman1, datos_iman2, datos_iman3, datos_iman4], keys=['1', '2', '3', '4'])
datos_totales = datos_totales.reset_index(level=0).rename(columns={'level_0': 'iman'})
datos_totales['campo_militeslas'] = datos_totales['campo_militeslas'].apply(lambda x: abs(x))
datos_totales['campo_gauss'] = datos_totales['campo_gauss'].apply(lambda x: abs(x))
datos_totales['iman'] = datos_totales['iman'].astype('int')

datos_totales.head(4)

,iman,estado_digital,valor_sensor,voltaje,campo_gauss,campo_militeslas,distancia_metros
0,1,0,533,260.508,0.00543,0.00054,0.15
1,1,0,536,261.975,0.01357,0.00136,0.10
2,1,0,551,269.306,0.05430,0.00543,0.05
3,1,0,586,286.413,0.14934,0.01493,0.01


In [14]:
# Gráfica de campos magnéticos en relación con la distancia
fig = px.line(datos_totales, x='distancia_metros', y='campo_militeslas', color='iman',
              labels={'distancia_metros': 'Distancia (metros)', 'campo_militeslas': 'Campo (militeslas)'},
              title='Gráfico de Campos Magnéticos')

fig.update_layout(
    legend_title='Imán',
    xaxis_title='Distancia del sensor al imán (m)',
    yaxis_title='Campo magnético (mT)',
    width=800,  
    height=500,
)

fig.show()

Ajustes con curve_fit de los datos con la función inversa $\frac{1}{x}$ y métricas de evaluación

In [15]:
# Funciones de ajuste
def func1(x, a, b):
    return a / x + b

def func2(x, a, b):
    return a / (x**2) + b

def func3(x, a, b):
    return a / (x**3) + b

# Crear un DataFrame para almacenar los resultados
resultados_df = pd.DataFrame(columns=['Imán', 'Función', 'R²', 'MSE', 'MAE'])

# Funciones de ajuste
funciones = [func1, func2, func3]
funciones_nombres = ['Función 1/x', 'Función 1/x^2', 'Función 1/x^3']

# Iterar sobre los imanes y funciones
for iman in [1, 2, 3, 4]:
    datos_iman = datos_totales[datos_totales['iman'] == iman]
    
    for funcion, nombre in zip(funciones, funciones_nombres):
        # Ajuste y predicciones
        popt, _ = curve_fit(funcion, datos_iman['distancia_metros'], datos_iman['campo_militeslas'])
        y_pred = funcion(datos_iman['distancia_metros'], *popt)
        
        # Cálculo de métricas
        r2 = r2_score(datos_iman['campo_militeslas'], y_pred)
        mse = mean_squared_error(datos_iman['campo_militeslas'], y_pred)
        mae = mean_absolute_error(datos_iman['campo_militeslas'], y_pred)
        
        resultados_df = resultados_df.append({'Imán': iman, 'Función': nombre, 'R²': r2, 'MSE': mse, 'MAE': mae}, ignore_index=True)

resultados_df.sort_values(by='Función')

,Imán,Función,R²,MSE,MAE
0,1,Función 1/x,0.959614,1.319139e-06,0.000967
3,2,Función 1/x,0.975689,4.642242e-07,0.000582
6,3,Función 1/x,0.917080,1.819931e-08,0.000117
9,4,Función 1/x,0.994224,1.623675e-08,0.000117
1,1,Función 1/x^2,0.913700,2.818869e-06,0.001368
4,2,Función 1/x^2,0.905349,1.807374e-06,0.001140
7,3,Función 1/x^2,0.817633,4.002596e-08,0.000173
10,4,Función 1/x^2,0.998581,3.987331e-09,0.000053
2,1,Función 1/x^3,0.899244,3.291059e-06,0.001466
5,2,Función 1/x^3,0.842458,3.008284e-06,0.001412


Descripción estadística del comportamiento según las métricas de los ajustes de las funciones inversas

In [16]:
resultados_df.groupby(by='Función').describe()

R²                                                    \
              count      mean       std       min       25%       50%   
Función                                                                 
Función 1/x     4.0  0.961652  0.032908  0.917080  0.948981  0.967652   
Función 1/x^2   4.0  0.908816  0.073955  0.817633  0.883420  0.909524   
Función 1/x^3   4.0  0.867486  0.100208  0.744817  0.818048  0.870851   

                                    MSE                ...                \
                    75%       max count          mean  ...           75%   
Función                                                ...                 
Función 1/x    0.980323  0.994224   4.0  4.544499e-07  ...  6.779530e-07   
Función 1/x^2  0.934920  0.998581   4.0  1.167564e-06  ...  2.060248e-06   
Función 1/x^3  0.920289  0.983427   4.0  1.600484e-06  ...  3.078978e-06   

                          MAE                                          \
                    max count      mean       std       min       25%   
Función                                                                 
Función 1/x    0.000001   4.0  0.000446  0.000411  0.000117  0.000117   
Función 1/x^2  0.000003   4.0  0.000683  0.000667  0.000053  0.000143   
Función 1/x^3  0.000003   4.0  0.000812  0.000724  0.000172  0.000193   

                                             
                    50%       75%       max  
Función                                      
Función 1/x    0.000350  0.000679  0.000967  
Función 1/x^2  0.000656  0.001197  0.001368  
Función 1/x^3  0.000806  0.001425  0.001466  

[3 rows x 24 columns]

Gráfica de ajustes de la forma a los datos de ditancia del sensor contra el campo medido en militeslas (mT)

In [17]:
fig = make_subplots(rows=1, cols=3, subplot_titles=['Ajuste de la forma 1/x', 'Ajuste de la forma 1/x^2', 'Ajuste de la forma 1/x^3'])

# Paletas de colores
colores_1 = px.colors.qualitative.Plotly
colores_2 = px.colors.qualitative.Dark2

# Definición de funciones inversas
def func1(x, a, b):
    return a / (x) + b

def func2(x, a, b):
    return a / (x**2) + b

def func3(x, a, b):
    return a / (x**3) + b

# Itera sobre los subplots y los imanes
for i, subplot_title, imanes, colores in zip([1, 2, 3], [''], [[1, 2, 3, 4]], [colores_1, colores_2]):
    for iman, color in zip(imanes, colores):
        
        datos_iman = datos_totales[datos_totales['iman'] == iman]

        # Ajuste polinomial
        popt, _ = curve_fit(func1, datos_iman['distancia_metros'], datos_iman['campo_militeslas'])
        a_opt, b_opt = popt

        # Definir la fórmula del polinomio ajustado
        polynomial_formula = f"{a_opt:.6f}/x {b_opt:.6f}"

        # Generar datos para la curva ajustada
        x_fit = np.linspace(datos_iman['distancia_metros'].min(), datos_iman['distancia_metros'].max(), 100)
        y_fit = func1(x_fit, *popt)

        # Agregar scatter plot de datos reales
        scatter_trace = px.scatter(datos_iman, x='distancia_metros', y='campo_militeslas', color='iman').update_traces(
            marker=dict(color=color)).data[0]

        # Agregar línea de ajuste a la subgráfica
        fit_trace = go.Scatter(x=x_fit, y=y_fit, mode='lines', name=f'Ajuste imán {iman}', line=dict(color=color, dash='dash'))
        fig.add_trace(scatter_trace, row=1, col=1)
        fig.add_trace(fit_trace, row=1, col=1)

for i, subplot_title, imanes, colores in zip([1, 2, 3], ['Imanes 1 y 2', 'Imanes 3 y 4'], [[1, 2, 3, 4]], [colores_1, colores_2]):
    for iman, color in zip(imanes, colores):
        
        datos_iman = datos_totales[datos_totales['iman'] == iman]

        # Ajuste polinomial
        popt, _ = curve_fit(func2, datos_iman['distancia_metros'], datos_iman['campo_militeslas'])
        a_opt, b_opt = popt

        # Definir la fórmula del polinomio ajustado
        polynomial_formula = f"{a_opt:.6f}/x^2 {b_opt:.6f}"

        # Generar datos para la curva ajustada
        x_fit = np.linspace(datos_iman['distancia_metros'].min(), datos_iman['distancia_metros'].max(), 100)
        y_fit = func2(x_fit, *popt)

        # Agregar scatter plot de datos reales
        scatter_trace = px.scatter(datos_iman, x='distancia_metros', y='campo_militeslas', color='iman').update_traces(
            marker=dict(color=color)).data[0]

        # Agregar línea de ajuste a la subgráfica
        fit_trace = go.Scatter(x=x_fit, y=y_fit, mode='lines', line=dict(color=color, dash='dash'), showlegend=False)
        fig.add_trace(scatter_trace, row=1, col=2)
        fig.add_trace(fit_trace, row=1, col=2)

for i, subplot_title, imanes, colores in zip([1, 2, 3], ['Imanes 1 y 2', 'Imanes 3 y 4'], [[1, 2, 3, 4]], [colores_1, colores_2]):
    for iman, color in zip(imanes, colores):
        
        datos_iman = datos_totales[datos_totales['iman'] == iman]

        # Ajuste polinomial
        popt, _ = curve_fit(func3, datos_iman['distancia_metros'], datos_iman['campo_militeslas'])
        a_opt, b_opt = popt

        # Definir la fórmula del polinomio ajustado
        polynomial_formula = f"{a_opt:.6f}/x^2 {b_opt:.6f}"

        # Generar datos para la curva ajustada
        x_fit = np.linspace(datos_iman['distancia_metros'].min(), datos_iman['distancia_metros'].max(), 100)
        y_fit = func3(x_fit, *popt)

        # Agregar scatter plot de datos reales
        scatter_trace = px.scatter(datos_iman, x='distancia_metros', y='campo_militeslas', color='iman').update_traces(
            marker=dict(color=color)).data[0]

        # Agregar línea de ajuste a la subgráfica
        fit_trace = go.Scatter(x=x_fit, y=y_fit, mode='lines', line=dict(color=color, dash='dash'), showlegend=False)
        fig.add_trace(scatter_trace, row=1, col=3)
        fig.add_trace(fit_trace, row=1, col=3)

fig.update_layout(
    title_text='Ajustes de la forma a los datos de ditancia del sensor contra el campo medido en militeslas (mT)',
    legend_title='Ajustes',
    xaxis_title='Distancia del sensor al imán (m)',
    yaxis_title='Campo magnético (mT)',
    xaxis2_title='Distancia del sensor al imán (m)',
    xaxis3_title='Distancia del sensor al imán (m)',
    width=1200,
    height=500,
)

# fig.write_image("ajustes x^-1.png")
fig.show()

Gráfica de ajustes polinomiales de segundo grado

In [20]:
fig = make_subplots(rows=1, cols=2, subplot_titles=['Imanes 1 y 2', 'Imanes 3 y 4'])

# Paletas de colores
colores_1 = px.colors.qualitative.Plotly
colores_2 = px.colors.qualitative.Set2

# Función de ajuste polinomial
def ajuste_polinomial(x, y, grado, color):
    coeficientes = np.polyfit(x, y, grado)
    polinomio = np.poly1d(coeficientes)
    formula = str(polinomio).replace('x', 'x^2', 1).split('\n')[-1]
    return polinomio, color, formula

for i, subplot_title, imanes, colores in zip([1, 2], ['Imán 1 y 2', 'Imán 3 y 4'], [[1, 2], [3, 4]], [colores_1, colores_2]):
    for iman, color in zip(imanes, colores):
        datos_iman = datos_totales[datos_totales['iman'] == iman]
        
        # Ajuste polinomial de grado 2
        polinomio, curva_color, formula = ajuste_polinomial(datos_iman['distancia_metros'], datos_iman['campo_militeslas'], grado=2, color=color)
        x_vals = np.linspace(min(datos_iman['distancia_metros']), max(datos_iman['distancia_metros']), 100)
        y_vals = polinomio(x_vals)
        
        # Agrega los datos ajustados como una línea adicional
        fig.add_trace(
            px.line(x=x_vals, y=y_vals, line_shape='linear')
            .update_traces(line=dict(color=curva_color))
            .data[0],
            row=1, col=i
        )
        
        # Agrega los datos originales como puntos
        fig.add_trace(
            go.Scatter(
                x=datos_iman['distancia_metros'], y=datos_iman['campo_militeslas'],
                mode='markers', marker=dict(color=color), name=f'Datos Originales Imán {iman}'),
            row=1, col=i
        )

fig.update_layout(
    title_text='Ajustes polinomiales de segundo grado para los datos de distancia en relación con el campo magnético',
    legend_title='Ajustes polinomiales',
    xaxis_title='Distancia del sensor al imán (m)',
    yaxis_title='Campo magnético (mT)',
    xaxis2_title='Distancia del sensor al imán (m)',
    width=1200,
    height=500,
)

# fig.write_image("ajustes_polinomiales.png")

fig.show()

In [22]:
fig = make_subplots(rows=1, cols=2, subplot_titles=['Imanes 1 y 2', 'Imanes 3 y 4'])

# Paletas de colores
colores_1 = px.colors.qualitative.Plotly
colores_2 = px.colors.qualitative.Set2

# Itera sobre los subplots y los imanes
for i, subplot_title, imanes, colores in zip([1, 2], ['Imán 1 y 2', 'Imán 3 y 4'], [[1, 2], [3, 4]], [colores_1, colores_2]):
    for iman, color in zip(imanes, colores):
        datos_iman = datos_totales[datos_totales['iman'] == iman]
        # Agrega las líneas
        fig.add_trace(
            px.line(datos_iman, x='distancia_metros', y='campo_militeslas', color='iman')
            .update_traces(showlegend=True, line=dict(color=color), name=f'Imán {iman}').data[0],
            row=1, col=i
        )

        # Agrega los puntos
        fig.add_trace(
            px.scatter(datos_iman, x='distancia_metros', y='campo_militeslas', color='iman')
            .update_traces(showlegend=False, marker=dict(color=color, size=8), name=f'Imán {iman}').data[0],
            row=1, col=i
        )

fig.update_layout(
    title_text='Distribución y tendencia de los datos de campo magnético por imán respecto a la distancia del sensor',
    legend_title='Imán',
    xaxis_title='Distancia del sensor al imán (m)',
    yaxis_title='Campo magnético (mT)',
    xaxis2_title='Distancia del sensor al imán (m)',
    xaxis_range=[0, 0.17],  
    yaxis_range=[0, 0.017], 
    yaxis2_range=[0, 0.006],
    xaxis2_range=[0, 0.06],
    width=1200,  
    height=600,  
)
# fig.write_image("distribucion_tendencia.png")

fig.show()